# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port blair,11.6667,92.7500,81.43,75,52,10.76,IN,1732216589
1,1,hotan,37.0997,79.9269,43.23,48,100,2.10,CN,1732216590
2,2,wailua homesteads,22.0669,-159.3780,75.99,87,20,6.91,US,1732216591
3,3,puerto baquerizo moreno,-0.9000,-89.6000,81.59,54,40,11.50,EC,1732216592
4,4,port-aux-francais,-49.3500,70.2167,39.60,82,87,38.97,TF,1732216593


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot

map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
        frame_width=800,
    frame_height=500,
    tiles='OSM',
    color='City',
    size='Humidity',
    hover_cols=['City', 'Country', 'Humidity'],
    alpha=0.6,
    title='City Map with Humidity Levels',
    scale = 1
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= 60) & (city_data_df["Max Temp"] <= 80) &
    (city_data_df["Humidity"] <= 60) &
    (city_data_df["Wind Speed"] < 5)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,san patricio,28.0170,-97.5169,73.35,16,0,4.09,US,1732216603
14,14,chonchi,-42.6128,-73.8092,65.05,40,55,2.57,CL,1732216605
44,44,betare oya,5.6000,14.0833,74.43,41,99,3.29,CM,1732216639
47,47,redlands,34.0556,-117.1825,77.40,15,0,1.01,US,1732216510
180,180,bibiani,6.4635,-2.3194,75.94,50,1,0.56,GH,1732216791
214,214,santa maria del oro,25.9333,-105.3667,71.64,21,14,4.29,MX,1732216831
246,246,alice springs,-23.7000,133.8833,65.75,26,0,0.00,AU,1732216781
298,298,mizdah,31.4451,12.9801,63.19,55,0,4.88,LY,1732216926
303,303,santa ana,33.7456,-117.8678,75.78,28,0,3.44,US,1732216931
345,345,post,33.1909,-101.3782,63.97,30,0,0.85,US,1732216980


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
12,san patricio,US,28.0170,-97.5169,16,
14,chonchi,CL,-42.6128,-73.8092,40,
44,betare oya,CM,5.6000,14.0833,41,
47,redlands,US,34.0556,-117.1825,15,
180,bibiani,GH,6.4635,-2.3194,50,
214,santa maria del oro,MX,25.9333,-105.3667,21,
246,alice springs,AU,-23.7000,133.8833,26,
298,mizdah,LY,31.4451,12.9801,55,
303,santa ana,US,33.7456,-117.8678,28,
345,post,US,33.1909,-101.3782,30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 25,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
san patricio - nearest hotel: No hotel found
chonchi - nearest hotel: Hotel Huildín
betare oya - nearest hotel: No hotel found
redlands - nearest hotel: Ayres Hotel Redlands
bibiani - nearest hotel: Addai's Hotel
santa maria del oro - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
mizdah - nearest hotel: No hotel found
santa ana - nearest hotel: Holiday Inn Express
post - nearest hotel: Hotel Garza Bed & Breakfast
crane - nearest hotel: No hotel found
ndele - nearest hotel: No hotel found
dinguiraye - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
essaouira - nearest hotel: Essaouira Wind Palace
tabalah - nearest hotel: No hotel found
merlo - nearest hotel: Hotel Contilo


,City,Country,Lat,Lng,Humidity,Hotel Name
12,san patricio,US,28.0170,-97.5169,16,No hotel found
14,chonchi,CL,-42.6128,-73.8092,40,Hotel Huildín
44,betare oya,CM,5.6000,14.0833,41,No hotel found
47,redlands,US,34.0556,-117.1825,15,Ayres Hotel Redlands
180,bibiani,GH,6.4635,-2.3194,50,Addai's Hotel
214,santa maria del oro,MX,25.9333,-105.3667,21,No hotel found
246,alice springs,AU,-23.7000,133.8833,26,Aurora Alice Springs
298,mizdah,LY,31.4451,12.9801,55,No hotel found
303,santa ana,US,33.7456,-117.8678,28,Holiday Inn Express
345,post,US,33.1909,-101.3782,30,Hotel Garza Bed & Breakfast


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    frame_width=800,
    frame_height=500,
    tiles='OSM',
    color='City',  # Color by city name for visual differentiation
    size='Humidity',
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'],
    alpha=0.6,
    title='Ideal Vacation Cities with Nearest Hotels and Humidity Levels',
    scale=1
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)